# DREAM5

In [ ]:
from pathlib import Path

RAW_DATA_ROOT = Path("../local_data/raw/syn2787209/Gene Network Inference")
PROCESSED_DATA_ROOT = Path("../local_data/processed/dream_five")
NETWORK_ID = 1

In [14]:
from fedgenie3.data.preprocessing.dream_five import preprocess_dream_five

preprocess_dream_five(RAW_DATA_ROOT, PROCESSED_DATA_ROOT)


Processing DREAM5 data from ../data/raw/syn2787209/Gene Network Inference to ../data/processed/dream_five


Processing DREAM5 data: 100%|██████████| 3/3 [00:06<00:00,  2.23s/network]


In [15]:
from fedgenie3.genie3.run import run

run(PROCESSED_DATA_ROOT, NETWORK_ID)

Running (non-federated) GRN inference for network 1 in ../data/processed/dream_five


Computing importances:   0%|          | 0/1643 [00:00<?, ?gene/s]